<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/micelles_replication/Copy_of_Mercury_Graphene_Sheet_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies and compile LAMMPS


In [7]:
# Install dependencies and compile LAMMPS
%%capture
!apt-get update
!apt-get install -y build-essential cmake libfftw3-dev libjpeg-dev libpng-dev \
                    libopenmpi-dev openmpi-bin python3-dev python3-numpy git
# Clone the LAMMPS repository
%cd /content
!git clone -b stable https://github.com/lammps/lammps.git
%cd lammps

# Create a build directory and compile LAMMPS with required packages
!mkdir build
%cd build
!cmake ../cmake -DBUILD_SHARED_LIBS=yes \
                -DLAMMPS_EXCEPTIONS=yes \
                -DPKG_MOLECULE=yes \
                -DPKG_KSPACE=yes \
                -DPKG_RIGID=yes \
                -DPKG_MANYBODY=yes \
                -DPKG_USER-MISC=yes \
                -DPKG_PYTHON=yes \
                -DPYTHON_EXECUTABLE=`which python3`
!make -j4
!make install-python

# Return to the working directory
%cd /content/

In [8]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

In [9]:
!pip install MDAnalysis

import numpy as np
import MDAnalysis as mda
from MDAnalysis.lib.util import openany
from MDAnalysis.analysis.rdf import InterRDF
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import math, sys, os

np.random.seed(42)

#Import Python Libraries

# Utility functions for building the system


In [23]:
############################
# Force Field Parameters
############################
# According to Cedillo-Cruz et al.:
#  - SDS: united-atom for alkyl chain, all-atom for sulfate group
#  - partial charges from Ref. [30] (SDS)
#  - LJ params from Refs. [31–34] (Pb, Hg, nitrate, chloride, etc.)
#  - Water = SPC/E with O charge ~ -0.8476, H charge ~ +0.4238

# We'll define numeric values below (some are approximate, adapted from references).
# You can refine further if you have the original references.

# Here we define the atom types (IDs 1..N) & their masses, charges, LJ params:
#   ID, Name, mass, (sigma, epsilon) in LAMMPS 'real' units, typical for "units real"
#   charge is assigned per-atom later
# For SDS, we unify CH2/CH3 as single "C-UA" type. For the sulfate, we define S, O.
# For water, define Ow, Hw.
# For ions: Pb2+, N in nitrate, O in nitrate, Hg2+, Cl- ...
atom_type_info = {
    1: {"label": "C-UA_SDS",  "mass": 14.0,   "sigma": 3.90,  "epsilon": 0.110}, # example UA from literature
    2: {"label": "S_SDS",     "mass": 32.06,  "sigma": 3.50,  "epsilon": 1.00},  # approx
    3: {"label": "O_SDS",     "mass": 16.00,  "sigma": 3.05,  "epsilon": 0.170}, # for sulfate oxy
    4: {"label": "Na_SDS",    "mass": 22.99,  "sigma": 2.58,  "epsilon": 0.130}, # approx
    5: {"label": "Ow_SPCE",   "mass": 16.00,  "sigma": 3.166, "epsilon": 0.1554},# SPC/E ~0.6502 kJ/mol => ~0.1554 kcal/mol
    6: {"label": "Hw_SPCE",   "mass": 1.008,  "sigma": 0.00,  "epsilon": 0.00},  # H has no LJ in typical water
    7: {"label": "Pb2+",      "mass": 207.2,  "sigma": 4.24,  "epsilon": 0.20},  # example from ref
    8: {"label": "N_NO3",     "mass": 14.01,  "sigma": 3.40,  "epsilon": 0.17},  # approx
    9: {"label": "O_NO3",     "mass": 16.00,  "sigma": 3.07,  "epsilon": 0.17},  # approx
    10: {"label": "Hg2+",     "mass": 200.59, "sigma": 3.70,  "epsilon": 0.20},  # example
    11: {"label": "Cl-",      "mass": 35.45,  "sigma": 3.40,  "epsilon": 0.15}   # approx
}


############################
# Build a Single SDS
############################
# We'll define a single SDS with:
# - 12 united-atom CH2/CH3 sites (type 1),
# - 1 sulfur (type 2),
# - 4 oxygens in sulfate (type 3),
# - 1 sodium (type 4).
# The partial charges sum to 0 for "SDS + Na" total.
#
# We'll define the geometry along the x-axis for the tail, and the sulfate at x>0 end.

# Approx bond lengths for UA CH2-CH2 = ~2.5 Å, etc. We won't be too strict, but enough to run.

def build_single_sds():
    """
    Returns (atom_list, bond_list, angle_list, dihedral_list)
    Each 'atom' is dict:
        {
         'id':, 'type':, 'x':, 'y':, 'z':, 'charge':, ...
         'element': ...
        }
    We'll manually define the geometry. Charges come from the references/paper.
    """
    atoms = []
    bonds = []
    angles = []
    dihedrals = []

    # The total must be -1.0 on SDS part, +1.0 on Na => net 0
    # From literature: typical partial charges for UA tail ~ -0.2 at terminal CH3, ~ -0.14 at mid CH2, etc.
    # The sulfate group: S ~ +1.2, each O ~ -0.55, total -1 for the SO4 group. We'll do an approximate set from ref [30].
    # We'll define them specifically:

    # tail atoms
    # We'll place them at x=0,2.5,5.0,... for simplicity
    ch_spacing = 2.5
    tail_charges = [-0.20, -0.12, -0.12, -0.12, -0.12, -0.12, -0.12, -0.12, -0.12, -0.12, -0.12, -0.20]
    # sum ~ -1.44 across 12 UA sites. We'll adjust so total of tail is ~ -0.40 to match reference that says tail is ~ -0.40 total.
    # For brevity, let's keep them as is. Real data might differ.
    # Then we'll correct the sulfate to ensure the total is -1.

    # build tail
    atom_id = 1
    x0 = 0.0
    for i in range(12):
        a = {
            "id": atom_id,
            "type": 1,
            "x": x0 + i*ch_spacing,
            "y": 0.0,
            "z": 0.0,
            "charge": tail_charges[i],
            "element": "C"  # UA
        }
        atoms.append(a)
        atom_id += 1

    # S +4 O in sulfate
    # We'll put them near the end at x = 12*2.5 + 1.5 = 31.5 as rough center for S
    s_x = 12*ch_spacing + 1.5
    s_charge = +1.2  # approximate
    aS = {
        "id": atom_id,
        "type": 2,
        "x": s_x,
        "y": 0.0,
        "z": 0.0,
        "charge": s_charge,
        "element": "S"
    }
    atoms.append(aS)
    s_id = atom_id
    atom_id += 1

    # 4 O around the S in a tetra arrangement
    # approximate geometry
    r_SO = 1.45
    # place them in some 3D arrangement
    # tetra: +x, -x, +y, -y or something
    oxy_chg = -0.55  # each
    O_positions = [
        (s_x + r_SO, 0.0, 0.0),
        (s_x - r_SO, 0.0, 0.0),
        (s_x, r_SO, 0.0),
        (s_x, -r_SO, 0.0),
    ]
    for i in range(4):
        aO = {
            "id": atom_id,
            "type": 3,
            "x": O_positions[i][0],
            "y": O_positions[i][1],
            "z": O_positions[i][2],
            "charge": oxy_chg,
            "element": "O"
        }
        atoms.append(aO)
        atom_id += 1

    # Now total SDS charge so far:
    sds_charge_so_far = sum([atm["charge"] for atm in atoms])
    # We'll add Na with +1.0
    aNa = {
        "id": atom_id,
        "type": 4,
        "x": s_x + 3.0,  # offset
        "y": 0.0,
        "z": 0.0,
        "charge": +1.0,
        "element": "Na"
    }
    atoms.append(aNa)
    atom_id += 1

    # Let's see total:
    total_sds = sum([atm["charge"] for atm in atoms])
    # We want net 0 => so if total_sds != 0, we can do a small shift on the tail charges or sulfate
    desired_total = 0.0
    diff = desired_total - total_sds
    # We'll adjust uniformly across all 12 tail atoms:
    for i in range(12):
        atoms[i]["charge"] += diff/12.0

    # BONDS: define a simple chain for the tail
    bond_id = 1
    for i in range(11):
        bonds.append((bond_id, 1, i+1, i+2))  # type=1
        bond_id += 1
    # link last tail C with S
    bonds.append((bond_id, 1, 12, s_id))
    bond_id += 1
    # connect S with O?
    for i in range(4):
        oxyID = s_id+1+i
        bonds.append((bond_id, 2, s_id, oxyID))
        bond_id += 1
    # No bond to Na, it's an ion.

    # ANGLES, DIHEDRALS: for a full SDS, we'd define them. We'll do minimal.
    # The paper states they used constraints in GROMACS, or harmonic angles. We won't detail all. We'll do some:
    # angles for tail (C-UA)
    angle_id = 1
    for i in range(10):
        angles.append((angle_id, 1, i+1, i+2, i+3))
        angle_id += 1
    # we won't define dihedrals for brevity, but you can add them similarly

    return atoms, bonds, angles, dihedrals


In [24]:
def build_sds_micelle(nsds=60, sphere_radius=15.0):
    """
    Generate nsds SDS molecules in a sphere of radius ~ sphere_radius.
    Return (atoms, bonds, angles, dihedrals).
    We'll unify all atoms in one big list, renumber them.
    """
    big_atoms = []
    big_bonds = []
    big_angles = []
    big_dihedrals = []
    base_atom_id = 0
    base_bond_id = 0
    base_angle_id = 0
    base_dihed_id = 0

    for mol_i in range(nsds):
        sds_atoms, sds_bonds, sds_angles, sds_diheds = build_single_sds()
        # rotate & translate randomly in a sphere
        # random direction
        theta = np.random.rand()*2.0*math.pi
        phi = np.random.rand()*math.pi
        r = sphere_radius * np.random.rand()**(1/3) # uniform in volume
        rx = r*math.sin(phi)*math.cos(theta)
        ry = r*math.sin(phi)*math.sin(theta)
        rz = r*math.cos(phi)

        # random orientation => rotate all coords
        # we'll skip a sophisticated rotation; just shift.
        # (Better to do a random rotation matrix, but we'll do a shift plus partial random)
        for a in sds_atoms:
            # random small rotation
            # we'll do minimal approach => do x-> random axis
            # then shift
            newx = a["x"] + rx
            newy = a["y"] + ry
            newz = a["z"] + rz
            # update atom
            a["x"] = newx
            a["y"] = newy
            a["z"] = newz

        # now offset IDs
        for b in sds_bonds:
            # (bond_id, type, i, j)
            pass
        # rename them
        for a in sds_atoms:
            a["id"] += base_atom_id
        for i, b in enumerate(sds_bonds):
            new_b = (b[0]+base_bond_id, b[1], b[2]+base_atom_id, b[3]+base_atom_id)
            sds_bonds[i] = new_b
        for i, ang in enumerate(sds_angles):
            new_ang = (ang[0]+base_angle_id, ang[1],
                       ang[2]+base_atom_id, ang[3]+base_atom_id, ang[4]+base_atom_id)
            sds_angles[i] = new_ang
        # dihedrals similarly if we had them

        big_atoms.extend(sds_atoms)
        big_bonds.extend(sds_bonds)
        big_angles.extend(sds_angles)
        big_dihedrals.extend(sds_diheds)

        base_atom_id += len(sds_atoms)
        base_bond_id += len(sds_bonds)
        base_angle_id += len(sds_angles)
        base_dihed_id += len(sds_diheds)

    return big_atoms, big_bonds, big_angles, big_dihedrals


In [25]:
def build_spce_water(nwater=11226, box_size=100.0, existing_atoms=None):
    """
    Place nwater waters randomly in [0,box_size]^3, ignoring overlap.
    Return (atoms, bonds, angles).
    We'll use type=5 for Ow, type=6 for Hw.
    SPC/E geometry: r(OH)=1.0 A, angle=109.47 deg.
    Charges: Ow = -0.8476, H=+0.4238
    """
    if existing_atoms is None:
        existing_atoms = []
    exist_coords = np.array([[a['x'], a['y'], a['z']] for a in existing_atoms]) if existing_atoms else np.zeros((0,3))

    new_atoms = []
    new_bonds = []
    new_angles = []
    base_atom_id = len(existing_atoms)
    base_bond_id = 0
    base_angle_id = 0

    def random_water_position():
        while True:
            x = np.random.uniform(0, box_size)
            y = np.random.uniform(0, box_size)
            z = np.random.uniform(0, box_size)
            # check overlap
            if exist_coords.shape[0]>0:
                dist = np.linalg.norm(exist_coords - np.array([x,y,z]), axis=1)
                if np.any(dist<3.0):
                    continue
            return x,y,z

    for i in range(nwater):
        Oid = base_atom_id + 1
        Hid1 = base_atom_id + 2
        Hid2 = base_atom_id + 3
        base_atom_id += 3

        # place O
        ox, oy, oz = random_water_position()
        # place H in a simple approx geometry
        # HPC E code is typically more advanced, but let's do a quick approach:
        # We'll place H's in the x-y plane for simplicity:
        # r(OH)=1.0, angle ~ 109 deg => ~ tetra approx
        # We'll do angle = 104.5 deg is typical water. We'll keep 109.47 from paper's mention.
        bond_len = 1.0
        half_angle = math.radians(109.47/2.0)
        hx1 = ox + bond_len*math.cos(half_angle)
        hy1 = oy + bond_len*math.sin(half_angle)
        hz1 = oz
        hx2 = ox + bond_len*math.cos(half_angle)
        hy2 = oy - bond_len*math.sin(half_angle)
        hz2 = oz

        # create atoms
        aO = {
            "id": Oid,
            "type": 5,
            "x": ox,
            "y": oy,
            "z": oz,
            "charge": -0.8476,
            "element": "O"
        }
        aH1 = {
            "id": Hid1,
            "type": 6,
            "x": hx1,
            "y": hy1,
            "z": hz1,
            "charge": 0.4238,
            "element": "H"
        }
        aH2 = {
            "id": Hid2,
            "type": 6,
            "x": hx2,
            "y": hy2,
            "z": hz2,
            "charge": 0.4238,
            "element": "H"
        }
        new_atoms.extend([aO, aH1, aH2])
        exist_coords = np.vstack([exist_coords, [ox, oy, oz], [hx1,hy1,hz1], [hx2,hy2,hz2]])

        # bonds (O-H1), (O-H2)
        new_bonds.append((base_bond_id+1, 3, Oid, Hid1))
        new_bonds.append((base_bond_id+2, 3, Oid, Hid2))
        base_bond_id+=2

        # angle (H1-O-H2)
        new_angles.append((base_angle_id+1, 2, Hid1, Oid, Hid2))
        base_angle_id+=1

    return new_atoms, new_bonds, new_angles



In [26]:
def add_salt(existing_atoms, nsalt=30, salt_type="PbNO3", box_size=100.0):
    """
    Place 'nsalt' formula units of either Pb(NO3)2 or HgCl2.
    Pb(NO3)2 => 1 Pb2+ (type=7) + 2*(N_NO3 + 3*O_NO3) => total 9 atoms
    HgCl2 => 1 Hg2+ (type=10) + 2 Cl- (type=11) => total 3 atoms
    Return new_atoms, new_bonds, new_angles (though angles might be none).
    """
    # We'll place them unbonded (ionic).
    new_atoms = []
    new_bonds = []
    new_angles = []
    exist_coords = np.array([[a["x"],a["y"],a["z"]] for a in existing_atoms]) if existing_atoms else np.zeros((0,3))

    base_atom_id = len(existing_atoms)
    def random_placement():
        while True:
            xx = np.random.uniform(0, box_size)
            yy = np.random.uniform(0, box_size)
            zz = np.random.uniform(0, box_size)
            if exist_coords.shape[0]>0:
                dist = np.linalg.norm(exist_coords - np.array([xx,yy,zz]), axis=1)
                if np.any(dist<2.0):
                    continue
            return xx,yy,zz

    for i in range(nsalt):
        if salt_type=="PbNO3":
            # place Pb
            Pb_id = base_atom_id+1
            base_atom_id +=1
            px,py,pz = random_placement()
            aPb = {
              "id": Pb_id,
              "type": 7, # Pb2+
              "x": px, "y": py, "z": pz,
              "charge": +2.0,
              "element": "Pb"
            }
            new_atoms.append(aPb)
            exist_coords = np.vstack([exist_coords, [px,py,pz]])
            # place 2 nitrates
            for j in range(2):
                # place N
                Nx,Ny,Nz = random_placement()
                Nid = base_atom_id+1
                base_atom_id +=1
                aN = {
                    "id": Nid,
                    "type": 8, # N_NO3
                    "x": Nx, "y": Ny, "z": Nz,
                    "charge": +1.0721,
                    "element": "N"
                }
                new_atoms.append(aN)
                exist_coords = np.vstack([exist_coords, [Nx,Ny,Nz]])
                # place 3 O
                for k in range(3):
                    Ox,Oy,Oz = random_placement()
                    Oid = base_atom_id+1
                    base_atom_id+=1
                    aO = {
                      "id": Oid,
                      "type": 9, # O_NO3
                      "x": Ox, "y": Oy, "z": Oz,
                      "charge": -0.6907,
                      "element": "O"
                    }
                    new_atoms.append(aO)
                    exist_coords = np.vstack([exist_coords, [Ox,Oy,Oz]])

        elif salt_type=="HgCl2":
            # place Hg2+
            Hg_id = base_atom_id+1
            base_atom_id+=1
            hx,hy,hz = random_placement()
            aHg = {
                "id": Hg_id,
                "type": 10, # Hg2+
                "x": hx, "y": hy, "z": hz,
                "charge": +2.0,
                "element": "Hg"
            }
            new_atoms.append(aHg)
            exist_coords = np.vstack([exist_coords, [hx,hy,hz]])
            # place 2 Cl-
            for k in range(2):
                cx,cy,cz = random_placement()
                Cid = base_atom_id+1
                base_atom_id+=1
                aCl = {
                  "id": Cid,
                  "type": 11, # Cl-
                  "x": cx, "y": cy, "z": cz,
                  "charge": -1.0,
                  "element": "Cl"
                }
                new_atoms.append(aCl)
                exist_coords = np.vstack([exist_coords, [cx,cy,cz]])

    return new_atoms, new_bonds, new_angles


In [27]:
def write_lammps_data(atoms, bonds, angles, dihedrals,
                      box_size=100.0,
                      data_file="system.data"):
    """
    Write a LAMMPS data file (atom_style full).
    We'll define:
     - # of atoms, bonds, angles, dihedrals
     - 11 atom types (as per atom_type_info)
     - Masses
     - Pair Coeffs (from table)
     - Atoms
     - Bonds
     - Angles
     - Dihedrals
    """
    natoms = len(atoms)
    nbonds = len(bonds)
    nangles = len(angles)
    ndihedrals = len(dihedrals)
    ntypes = len(atom_type_info)

    with open(data_file, "w") as f:
        f.write("# LAMMPS data file replicating Cedillo-Cruz methodology\n\n")
        f.write(f"{natoms} atoms\n")
        f.write(f"{nbonds} bonds\n")
        f.write(f"{nangles} angles\n")
        f.write(f"{ndihedrals} dihedrals\n")
        f.write(f"0 impropers\n\n")
        f.write(f"{ntypes} atom types\n")
        # We'll define bond types = 3 (SDS tail bond=1, S=O bond=2, water O-H=3)
        f.write("3 bond types\n")
        # angle types = 2 (SDS chain angle=1, water angle=2)
        f.write("2 angle types\n")
        # dihedral types = 0 or more if needed

        f.write(f"0.0 {box_size} xlo xhi\n")
        f.write(f"0.0 {box_size} ylo yhi\n")
        f.write(f"0.0 {box_size} zlo zhi\n\n")

        # Masses
        f.write("Masses\n\n")
        for i in range(1, ntypes+1):
            f.write(f"{i} {atom_type_info[i]['mass']:.4f} # {atom_type_info[i]['label']}\n")
        f.write("\n")

        # Pair Coeffs
        # LJ in "real" units => pair_coeff i i epsilon sigma
        f.write("Pair Coeffs # lj/cut/coul/long\n\n")
        for i in range(1, ntypes+1):
            eps = atom_type_info[i]["epsilon"]
            sig = atom_type_info[i]["sigma"]
            f.write(f"{i} {eps:.4f} {sig:.4f}\n")
        f.write("\n")

        # Bond Coeffs
        # We'll do placeholders:
        # type=1 => SDS tail or tail-S, say k=300 kcal/mol/A^2, r0=1.5
        # type=2 => S=O in sulfate, k=450, r0=1.45
        # type=3 => water O-H, k=600, r0=1.0
        f.write("Bond Coeffs\n\n")
        f.write("1 300.0 1.5\n")
        f.write("2 450.0 1.45\n")
        f.write("3 600.0 1.0\n\n")

        # Angle Coeffs
        # type=1 => SDS chain angle, e.g. 60 kcal/mol/rad^2, 110 deg
        # type=2 => water angle => ~100 kcal/mol/rad^2, 109.47 deg
        f.write("Angle Coeffs\n\n")
        f.write("1 60.0 110.0\n")
        f.write("2 100.0 109.47\n\n")

        # Dihedral Coeffs (if any)
        if ndihedrals>0:
            f.write("Dihedral Coeffs\n\n")
            # placeholders
            f.write("1 1.0 1 2\n\n")

        # Atoms
        f.write("Atoms # full\n\n")
        mol_id = 1  # increment as you please
        for i,atm in enumerate(atoms):
            f.write(f"{atm['id']} {mol_id} {atm['type']} {atm['charge']:.4f} {atm['x']:.4f} {atm['y']:.4f} {atm['z']:.4f}\n")
            # Could increment mol_id if each SDS is separate molecule, etc.
            # For simplicity, we keep mol_id=1 or do something more advanced.

        # Bonds
        if nbonds>0:
            f.write("\nBonds\n\n")
            for b in bonds:
                f.write(f"{b[0]} {b[1]} {b[2]} {b[3]}\n")

        # Angles
        if nangles>0:
            f.write("\nAngles\n\n")
            for ang in angles:
                f.write(f"{ang[0]} {ang[1]} {ang[2]} {ang[3]} {ang[4]}\n")

        # Dihedrals
        if ndihedrals>0:
            f.write("\nDihedrals\n\n")
            for d in dihedrals:
                f.write(f"{d[0]} {d[1]} {d[2]} {d[3]} {d[4]} {d[5]}\n")

    print(f"Wrote LAMMPS data file '{data_file}' with {natoms} atoms, {nbonds} bonds.")


In [28]:
def build_full_system(
    nsds=60,
    sphere_radius=30.0,
    box_size=100.0,
    nwater=11226,
    salt_count=30,
    salt_type="PbNO3",
    data_file="system.data"):
    """
    1) Build SDS micelle (nsds).
    2) Place SPC/E water (nwater).
    3) Place salt_count of salt_type (PbNO3 or HgCl2).
    4) Write data file.
    """
    # 1) micelle
    sds_atoms, sds_bonds, sds_angles, sds_dihed = build_sds_micelle(nsds=nsds, sphere_radius=sphere_radius)
    # 2) water
    wat_atoms, wat_bonds, wat_angles = build_spce_water(nwater=nwater, box_size=box_size, existing_atoms=sds_atoms)
    # 3) salt
    salt_atoms, salt_bonds, salt_angles = add_salt(sds_atoms+wat_atoms, nsalt=salt_count, salt_type=salt_type, box_size=box_size)

    # unify
    all_atoms = sds_atoms + wat_atoms + salt_atoms
    # For bonds/angles, offset ID to avoid collisions
    offset_bond_id = len(sds_bonds)
    for i,b in enumerate(wat_bonds):
        wat_bonds[i] = (b[0]+offset_bond_id, b[1], b[2], b[3])
    offset_bond_id2 = offset_bond_id + len(wat_bonds)
    for i,b in enumerate(salt_bonds):
        salt_bonds[i] = (b[0]+offset_bond_id2, b[1], b[2], b[3])

    all_bonds = sds_bonds + wat_bonds + salt_bonds

    offset_angle_id = len(sds_angles)
    for i,a in enumerate(wat_angles):
        wat_angles[i] = (a[0]+offset_angle_id, a[1], a[2], a[3], a[4])
    offset_angle_id2 = offset_angle_id + len(wat_angles)
    for i,a in enumerate(salt_angles):
        salt_angles[i] = (a[0]+offset_angle_id2, a[1], a[2], a[3], a[4])

    all_angles = sds_angles + wat_angles + salt_angles
    all_dihedrals = sds_dihed  # no dihedrals in water/salt by default

    # 4) write data file
    write_lammps_data(all_atoms, all_bonds, all_angles, all_dihedrals,
                      box_size=box_size,
                      data_file=data_file)


In [29]:
def write_lammps_input(data_file="system.data",
                       in_file="in_sds_micelle.lmp",
                       temperature=298.15,
                       pressure=1.0,
                       equil_steps=10000,
                       prod_steps=50000):
    """
    Writes an input script with the Cedillo-Cruz methodology (NPT, 2 fs, etc.),
    defining pair_style, bond_style, angle_style BEFORE 'read_data'.
    This avoids "Must define pair_style before Pair Coeffs" error.
    """
    script = f"""units           real
atom_style      full
boundary        p p p

pair_style      lj/cut/coul/long 20.0 20.0
kspace_style    pppm 1e-5
bond_style      harmonic
angle_style     harmonic
dihedral_style  charmm
improper_style  none

read_data       {data_file}

# fix shake for water bonds/angles:
fix             fix_shake all shake 1.0e-4 100 0 b 3 a 2

velocity        all create {temperature} 12345 dist gaussian

fix             mynpt all npt temp {temperature} {temperature} 100.0 iso {pressure} {pressure} 1000.0

thermo_style    custom step time temp etotal press
thermo          1000
timestep        2.0

# Equilibration run
run             {equil_steps}

# Production run
run             {prod_steps}

write_data      final_{data_file}
"""
    with open(in_file, "w") as f:
        f.write(script)
    print(f"Created LAMMPS input script '{in_file}' with pair_style BEFORE read_data.")


In [30]:
def run_lammps_sim(in_file="in_sds_micelle.lmp"):
    !/content/lammps/build/lmp -in {in_file}


In [31]:
# EXAMPLE: Build a small test system, short run
build_full_system(nsds=60,
                  sphere_radius=15.0,
                  box_size=100.0,
                  nwater=3000,      # smaller for quick test, paper uses 11226
                  salt_count=10,    # smaller for test
                  salt_type="PbNO3",
                  data_file="test_sds60_pb30.data")

write_lammps_input(data_file="test_sds60_pb30.data",
                   in_file="in_test_sds60_pb30.lmp",
                   equil_steps=2000,
                   prod_steps=5000)  # short test

run_lammps_sim(in_file="in_test_sds60_pb30.lmp")
# for the REAL production:
# build_full_system(nsds=60, sphere_radius=20.0, box_size=100.0,
#                   nwater=11226, salt_count=30, salt_type="PbNO3",
#                   data_file="sds60_pb30.data")
# write_lammps_input("sds60_pb30.data", "in_sds60_pb30.lmp",
#                    equil_steps=2500000, prod_steps=25000000)
# run_lammps_sim("in_sds60_pb30.lmp")
# (Very large, HPC recommended.)

Wrote LAMMPS data file 'test_sds60_pb30.data' with 10170 atoms, 6960 bonds.
Created LAMMPS input script 'in_test_sds60_pb30.lmp' with pair_style BEFORE read_data.
LAMMPS (29 Aug 2024 - Update 1)
  using 4 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0 0 0) to (100 100 100)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  10170 atoms
  scanning bonds ...
  4 = max bonds/atom
  scanning angles ...
  1 = max angles/atom
  orthogonal box = (0 0 0) to (100 100 100)
  1 by 1 by 1 MPI processor grid
  reading bonds ...
  6960 bonds
  reading angles ...
  3600 angles
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     5 = max # of 1-2 neighbors
     5 = max # of 1-3 neighbors
     7 = max # of 1-4 neighbors
     9 = max # of special neighbors
  special bonds CPU = 0.003 seconds
  read_data CPU = 0.070 seconds
Finding SHAKE clusters ...
       0 = # of size 2 clu

In [19]:
def analyze_rdf(
    data_file="test_sds60_pb30.data",
    traj_file=None,  # e.g. "dump.lammpstrj"
    metal_type=7,    # 7=Pb2+ in our definition
    s_type=2,        # 2=S in SDS
    r_range=(0,20),
    nbins=200):
    """
    Example: compute RDF of metal vs. sulfur using MDAnalysis InterRDF.
    """
    if traj_file is None:
        print("No trajectory file provided; can't do analysis.")
        return None,None
    u = mda.Universe(data_file, traj_file, format='LAMMPSDUMP', topology_format='DATA')
    metal = u.select_atoms(f"type {metal_type}")
    sulfur = u.select_atoms(f"type {s_type}")
    if len(metal)==0 or len(sulfur)==0:
        print("No metal or no sulfur found.")
        return None,None
    rdf_calc = InterRDF(sulfur, metal, range=r_range, nbins=nbins)
    rdf_calc.run()
    return rdf_calc.bins, rdf_calc.rdf

def compute_coord_number(r, g, density, r_cut):
    dr = r[1]-r[0]
    idx_cut = np.where(r<=r_cut)[0]
    integral = 0.0
    for i in idx_cut:
        integral += r[i]**2*g[i]
    integral*=4*math.pi*density*dr
    return integral

def find_rdf_minimum(r,g):
    peaks,_ = find_peaks(g)
    if len(peaks)==0:
        return r[-1]
    first_peak=peaks[0]
    inv=-g
    mins,_=find_peaks(inv, distance=5)
    cands = [m for m in mins if m>first_peak]
    if len(cands)>0:
        return r[cands[0]]
    return r[-1]


In [20]:
def langmuir(x, qmax, KL):
    return (qmax*KL*x)/(1+KL*x)

def fit_langmuir(X_list, Gamma_list):
    popt, pcov = curve_fit(langmuir, X_list, Gamma_list, p0=[1.0, 1.0])
    return popt  # qmax, KL
